<a href="https://colab.research.google.com/github/olaviinha/SloppyButcher/blob/master/legacy/sloppyButcher_v0_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="garamond">Sloppy Butcher <font size="4">v0.06a</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/SloppyButcher" target="_blank"><font color="#999" size="4">Github</font></a></font>
<font size="3" color="#999">&copy; 2020 <a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font>

### Manual

Sloppy Butcher is a strange little audio chopper tool based on multislicing and randomizing large sets of audio files. It was made to run in [Google Colab environment](https://colab.research.google.com) (i.e. your browser) using [your Google Drive](https://drive.google.com/drive/my-drive) as its audio data source and storage. The butcher takes a directory of audio files from your Google Drive, chops it up into very short slices, shuffles it, and frankensteins it up back into a single new audio track, following your settings and patterns.

The core functionality is based on the capabilities of [SoX](https://sourceforge.net/p/sox/code/ci/master/tree/) and [FFmpeg](https://github.com/FFmpeg/FFmpeg).

<h3>Quick start</h3>

1.   Create a directory in your Google Drive called `sloppybutcher`
2.   Drop some longish audio files in it (e.g. loops or songs)
3.   Hit <i>Runtime > Run all</i> from the menu.

<h3>Manual</h3>

1.   Create a directory for Sloppy Butcher in your Google Drive (e.g. `butcher-testing`)
2.   Drop some audio files inside that directory.
3.   Type in the path of the directory in the `input_dir` field (e.g. `butcher-testing`)
4.   Check `oneshots` if that is what your sounds are. They are processed differently for more sensible results.<br><small><font color="#888">Mixing long sounds (e.g. loop-like sounds) and oneshots in the directory is probably quite useless.</font></small>
5.   Adjust all the self-explanatory settings. Avoid conflicting settings as elaborated above each such setting.
6.   Make sure Google Drive (Backup and Sync) has finished syncing those audio files you dopped in Drive.
7.   <b>Optional:</b> Scroll down to <i>Patterns</i> (right below Settings) and see how those things work. Remember to check `use_patterns` in settings in this case.
8.   Hit <i>Runtime > Run all</i> from the menu.
9.   Wait. Depending on your source material, settings and patterns, processing may take a minute.
10.   Process is finished when an audio player appears at the bottom.
11.   Hit <i>Runtime > Run all</i> after every time you make changes to regenerate output.

### Editables

In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/tmp
!gsutil -q -m cp -R gs://olaviinha/slicer /content/tmp

In [ ]:
#@title Settings
#@markdown <pre>Audio source material</pre>
input_dir = "sloppybutcher" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
#handle_silence = "None" #@param ["None", "Trim_beginning_and_end", "Remove_everywhere"]
handle_silence = "None"
#@markdown <small>Check `oneshots` option if that is what your source sounds are. They are processed differently.</small>
oneshots = False #@param {type:"boolean"}
#@markdown <small>Checking `save_to_drive` will save every generated output file to your Drive. Uncheck if you just want to play around and listen to the previews before getting serious. Mounting Drive is still required for source audio.</small>
save_to_drive = False #@param {type:"boolean"}

#@markdown <pre>Presets</pre>
#@markdown <small>Selecting a `preset` will override all settings, even though nothing is visibly changed. Make sure `None` is selected if you want any of your knob twiddling to have any effect on the output.</small>
preset = "None" #@param ["None", "Entropy", "Breakcore", "Breakcore_oneshots_1", "Breakcore_oneshots_2", "Darkness", "Roboto", "Roboto_2", "Hell", "Deeper_Hell"]

#@markdown <pre>Basics</pre>
bpm = 83 #@param {type:"slider", min:60, max:200, step:1}
slices_per_beat = 4 #@param {type:"slider", min:1, max:4}

#@markdown <pre>Fancy</pre>
reverb = 33 #@param {type:"slider", min:0, max:100, step:1}
pitch_semitone = 6 #@param {type:"slider", min:-24, max:24, step:1}
reverse = True #@param {type:"boolean"}
release = 56 #@param {type:"slider", min:0, max:100}
#@markdown <small>Checking `tremolo` will cause `release` to be ignored.</small>
tremolo = False #@param {type:"boolean"}
#@markdown <small>Checking `crossfade` will cause `release`, `tremolo` and `use_patterns` to be ignored.</small>
crossfade = False #@param {type:"boolean"}
target_duration = 60 #@param {type:"slider", step:5, min:10, max:120}

#@markdown <pre>Use user-defined patterns</pre>
#@markdown <small>Checking `use_patterns` will cause `slices_per_beat` and `target_duration` to be ignored. Make sure to uncheck `crossfade` as it will cancel this option out.</small>
#@markdown <small>Patterns (`track[]`) are defined in the next section.</small>
use_patterns = False #@param {type:"boolean"}


# Fix certain things for now.
material_from = "googleDrive"
multitrack = False
fade_in_ms = 2
fade_out_ms = 2

drive_root = "/content/drive/My Drive/"
drive_dir = drive_root+input_dir
output_dir = drive_root+output_dir

import ntpath, random, os, math, librosa, datetime, string, psutil
import numpy as np
#import IPython
from os import listdir
from os.path import isfile, join
from glob import glob
from IPython.display import Image, Audio, display

fade_type = "h" #q=quarter of sin, h=half sin, t=linear, l=logarithmic, p=inverted parabola
ffmpeg_verbose = "-hide_banner -loglevel panic" # Empty for full output
sox_verbose = "-q" # -S for full output
vol = 0.6
maxArgs = 1000
min = 60
minutekb = 16479.4921875
allfiles = []
allConvFiles = []
allChopFiles = []
track_inputs = []
sliceLengths = []

dir_tmp = "/content/tmp/"
dir_tracks = dir_tmp+"tracks/"
dir_preview = dir_tmp+"previews/"
dir_waveform = dir_tmp+"waveform/"

if not os.path.isdir(dir_tmp):
  !mkdir {dir_tmp}
if not os.path.isdir(dir_tracks):
  !mkdir {dir_tracks}
if not os.path.isdir(dir_preview):
  !mkdir {dir_preview}
if not os.path.isdir(dir_waveform):
  !mkdir {dir_waveform}

if multitrack is True:
  dir_drop = dir_tracks
else:
  if not os.path.isdir(dir_tracks+"1"):
    !mkdir {dir_tracks}1
  dir_drop = dir_tracks+"1"

if material_from is "googleDrive":
  #from google.colab import drive
  #drive.mount('/content/drive')
  !cp -RT "{drive_dir}" {dir_drop}
elif material_from is "googleCloud":
  !gsutil -q -m cp -R {cloud_bucket} {dir_tmp}
else:
  ## TODO: all this
  from google.colab import files
  import zipfile
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('Uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)

# Presets
if preset is not "None":
  print('Using preset', preset)

if preset is "Entropy":
  handle_silence = "None"
  oneshots = False
  bpm = 280
  slices_per_beat = 4
  reverb = 80
  pitch_semitone = -4
  reverse = False
  release = 0
  tremolo = False
  crossfade = True
  target_duration = 60
  use_patterns = False

if preset is "Darkness":
  handle_silence = "None"
  oneshots = False
  bpm = 90
  slices_per_beat = 1
  reverb = 92
  pitch_semitone = -12
  reverse = True
  release = 0
  tremolo = False
  crossfade = True
  target_duration = 60
  use_patterns = False

if preset is "Roboto":
  handle_silence = "None"
  oneshots = False
  bpm = 200
  slices_per_beat = 4
  reverb = 13
  pitch_semitone = 12
  reverse = False
  release = 98
  tremolo = False
  crossfade = False
  target_duration = 60
  use_patterns = False

if preset is "Roboto_2":
  handle_silence = "None"
  oneshots = False
  bpm = 240
  slices_per_beat = 8
  reverb = 8
  pitch_semitone = 14
  reverse = False
  release = 100
  tremolo = False
  crossfade = False
  target_duration = 60
  use_patterns = False

if preset is "Hell":
  handle_silence = "None"
  oneshots = False
  bpm = 60
  slices_per_beat = 1
  reverb = 100
  pitch_semitone = -8
  reverse = True
  release = 0
  tremolo = False
  crossfade = True
  target_duration = 60
  use_patterns = False

if preset is "Deeper_Hell":
  handle_silence = "None"
  oneshots = False
  bpm = 20
  slices_per_beat = 1
  reverb = 100
  pitch_semitone = -18
  reverse = True
  release = 0
  tremolo = False
  crossfade = True
  target_duration = 60
  use_patterns = False

if preset is "Breakcore":
  handle_silence = "None"
  oneshots = False
  bpm = 180
  slices_per_beat = 4
  reverb = 8
  pitch_semitone = 2
  reverse = False
  release = 0
  tremolo = False
  crossfade = False
  target_duration = 60
  use_patterns = False

if preset is "Breakcore_oneshots_1":
  handle_silence = "None"
  oneshots = True
  bpm = 180
  slices_per_beat = 4
  reverb = 17
  pitch_semitone = -1
  reverse = False
  release = 0
  tremolo = False
  crossfade = False
  target_duration = 60
  use_patterns = False

if preset is "Breakcore_oneshots_2":
  handle_silence = "None"
  oneshots = True
  bpm = 184
  slices_per_beat = 4
  reverb = 22
  pitch_semitone = 2
  reverse = False
  release = 20
  tremolo = False
  crossfade = False
  target_duration = 60
  use_patterns = False

if preset is "Whistler":
  handle_silence = "None"
  oneshots = False
  bpm = 140
  slices_per_beat = 1
  reverb = 27
  pitch_semitone = 7
  reverse = True
  release = 0
  tremolo = True
  crossfade = False
  target_duration = 60
  use_patterns = False


# Adjustments
global_bpm = bpm
reverb_amount = reverb
reverb_damping = 100-reverb
pitch = pitch_semitone*100
fadeIn = fade_in_ms/1000
fadeOut = fade_out_ms/1000
approx_duration = target_duration
trackData = [None]

# Silence handler
if handle_silence is "Remove_everywhere":
  silence_handler = "silenceremove=1:0:-50dB"
elif handle_silence is "Trim_beginning_and_end":
  silence_handler = "silenceremove=start_periods=1:stop_periods=1:detection=peak"
else:
  silence_handler = ''

smooth_wave = True
if use_patterns is True and crossfade is False:
  smooth_wave = False

trackData = [None]

# Conflict warnings
def conflictWarn():
  if crossfade is True and use_patterns is True and tremolo is True:
    print('\nCONFLICT WARNING!\nYou have checked tremolo, crossfade, and use_patterns. Tremolo and patterns will be ignored. Uncheck crossfade to use patterns and tremolo.')
  elif crossfade is True and use_patterns is True and tremolo is False:
    print('\nCONFLICT WARNING!\nYou have checked both crossfade and use_patterns. Patterns (trackData[]) will be ignored. Uncheck crossfade to use patterns.')
  elif crossfade is True and use_patterns is False and tremolo is True:
    print('\nCONFLICT WARNING!\nYou have checked both crossfade and tremolo. Tremolo will be ignored. Uncheck crossfade to use tremolo.')

prefix = "frankenstein"
conflictWarn()

# trackData[0] = [
#   #title
#   'Demo',
#   #bpm
#   120,
#   # repeats per pattern
#   [2, 2, 2],
#   # slices, vols, beats
#   [[[1,1,1,1], [1,1,1,1], [0,0,0,0,0,0,0,0]],
#    [[2,2,2,2], [1,1,1,1], [0,0,0,0,0,0,0,0]],
#    [[4,4,4,4], [1,1,1,1], [0,0,0,0,0,0,0,0]]]
# ]

#### Patterns

```

Pattern explained:

  trackData[0] = [
     'Demo',           <- Title of this pattern
     120,              <- bpm of this pattern. 0 = use bpm from the slider in Settings^
     [4, 6, 1],        <- Repeat-list. How many times each bar of 4 beats is repeated. All 4 beat bars are listed right below.
     [[[1,1,1,1]],     <- Bar of 4 beats. In this bar each beat plays 1 slice. This bar is repeated 4 times.
      [[2,2,2,2]],     <- Same, except each beat plays 2 slices. This bar is repeated 6 times.
      [[4,1,2,4]]      <- Same, except the beats play different numbers of slices. This bar is played just 1 time.
                       <- Copy/paste more bars here indefinitely. 
     ]                    - Make sure each line ends with a comma (,) except the very last.    
   ]                      - Make sure there are as many numbers in the repeat-list as you have bars listed here.


```

In [ ]:
# The actual pattern:

trackData[0] = [
  'Demo',
  0,
  [4, 6, 8],
  [[[1,1,1,1]],
   [[2,2,2,2]],
   [[4,1,2,4]]
  ]  
]


### Setup

In [ ]:
#@title Install apps & clone repository
!pip install ffmpeg
!apt-get install sox
!apt-get install gnuplot
!git clone https://github.com/olaviinha/SloppyButcher.git

In [ ]:
#@title Definitions
def resetVars():
  allfiles = []
  allConvFiles = []
  allChopFiles = []
  track_inputs = []
  sliceLengths = []
  if multitrack is True:
    dir_drop = dir_tracks
  else:
    dir_drop = dir_tracks+"/1"

def resetFiles():
  if os.path.isdir(dir_tracks):
    !rm -r {dir_tmp}
  !mkdir {dir_tmp}
  !mkdir {dir_tracks}
  !mkdir {dir_preview}
  !mkdir {dir_waveform}
  if multitrack is True:
    dir_drop = dir_tracks
  else:
    if os.path.isdir(dir_tracks+"/1"):
      !mkdir {dir_tracks}/1
    dir_drop = dir_tracks+"/1"
  if material_from is "googleDrive":
    copyPath = stripPath(drive_dir)
    !cp -RT "{copyPath}" {dir_drop}
  else:
    !gsutil -q -m cp -R gs://olaviinha/ep {dir_tmp}
  !gsutil -q -m cp -R gs://olaviinha/slicer /content/tmp

def veryHardReset():
  print('Hose down the room...', end=' ')
  resetVars()
  resetFiles()
  print('Done.')

def playa(file):
  display(Audio(file))

def waveform(input):
  dir_dat = dir_tmp+"waveform"
  !sox "{input}" {dir_dat}/audio.dat
  !tail -n+3 {dir_dat}/audio.dat > {dir_dat}/audio_only.dat
  !gnuplot SloppyButcher/audio.gpi
  display(Image(dir_dat+'/audio.png'))

def wfAudio(play, show):
  waveform(show)
  playa(play)

def stripPath(path, *args):
  if path.endswith('/'):
    path = path[:-1]
  if args:
    path = path+"/"
  return path

def slug(s):
    valid_chars = "-_. %s%s" % (string.ascii_letters, string.digits)
    file = ''.join(c for c in s if c in valid_chars)
    file = file.replace(' ','_') # I don't like spaces in filenames.
    return file

def strList(s):
    str1 = " -v " + str(vol) + " " 
    return (str1.join(s)) 

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


def process(filelist, outputDir, type, oneshots, silence_handler):
  i=0
  for file in filelist:
    input = file
    if type is "convert":
      print('Marinating', path_leaf(input), '...', end=' ')
      output = outputDir+slug(path_leaf(basename(file)))+'.wav'
      #trim silence: silenceremove=start_periods=1:stop_periods=1:detection=peak
      #remove silence: silenceremove=1:0:-50dB
      pars = "pan=stereo|c0=c0|c1=c0"
      if silence_handler is not "":
        pars = pars+", "+silence_handler
      if oneshots is True:
        pars = pars+", apad=whole_len=48000"
        #!ffmpeg {ffmpeg_verbose} -y -i "{input}" -c:a pcm_s24le -ar 48000 -ac 2 -af {pars} "{output}"
      if reverse is True:
        pars = pars+", areverse"
      !ffmpeg {ffmpeg_verbose} -y -i "{input}" -c:a pcm_s24le -ar 48000 -ac 2 -af "{pars}" "{output}"
      print('Done.')
    if type is "pad":
      print(input, '...', end = ' ')
      output = outputDir+"co"+str(i)+".wav"
      !sox {input} {output} pad 0 3000
      print('Done.')
    if type is "trim":
      output = outputDir+"tr"+str(i)+".wav"
      !sox {sox_verbose} "{input}" {output} trim 0 {trim_dur}
    i+=1
  
#def chop(inputDir, outputDir, filelist, slice, fadeIn, fadeOut, mode):
def chop(inputDir, outputDir, filelist, slice, fadeIn, fadeOut, mode, tremolo):
  i = 0
  for file in filelist:
    print('Butchering', file, 'to prime cut '+str(slice)+'...', end='')
    input = str(inputDir)+str(file)
    output = str(outputDir)+basename(file)+str(i)+".wav"
    if mode is "smooth" or (mode is "slice" and tremolo is True):
      fadeIn = slice/2.8
      fadeOut = slice/2.8
      if tremolo is True:
        fadeType = "t"
      else:
        fadeType = "q"
      !sox {sox_verbose} "{input}" {output} trim 0 {slice} fade {fadeType} {fadeIn} -0 {fadeOut} : newfile : restart
    elif mode is "slice" and tremolo is False:
      if release > 0:
        fadeOut = (slice/100*release)-fadeIn
      !sox {sox_verbose} "{input}" {output} trim 0 {slice} fade {fadeIn} -0 {fadeOut} : newfile : restart
    else:
      !sox {sox_verbose} "{input}" {output} trim 0 {slice} fade 0 -0 {fadeOut}
    i+=1
    print('Done.')

def removeResidue(dir):
  #if oneshots is True:
  #  basesize = os.path.getsize(dir+"ch0.wav")
  #else:
  basesize = os.path.getsize(dir+sorted(os.listdir(dir))[0])
  #basesize = os.path.getsize(dir+"ch0001.wav")
  print('\nCheck if those warnings rottin\'n any chopperchups...')
  filelist = glob(dir+"*.wav")
  for file in filelist:
    if os.path.getsize(file) != basesize:
      print('Throw out', file)
      !rm {file}
  print('All prime chupachups.\n')

def getDuration(track, oneshots, slice):
  files = listFiles(track)
  duration = 0
  if oneshots is True:
    duration = len(files)*slice*4
  else:
    for file in files:
      duration += librosa.get_duration(filename=file)
  return duration

def listFiles(track):
  for ext in ('*.wav', '*.aiff', '*.flac', '*.mp3', '*.ogg'):
    allfiles.extend(glob(join(track, ext)))
  return allfiles

def trackInfo(track):
  print('Track title:', trackData[track-1][0])
  displayBpm = trackData[track-1][1]
  if displayBpm is 0:
    displayBpm = bpm
  print('BPM:', displayBpm)
  
def rndStr(length):
  letters = string.ascii_lowercase
  result_str = ''.join(random.choice(letters) for i in range(length))
  return result_str

def basename(path):
  filename = os.path.basename(path).strip().replace(" ", "_")
  filebase = os.path.splitext(filename)[0]
  return filebase

def stitchChunk(inputList, dir):
  newDir = dir+rndStr(8)
  !mkdir {newDir}
  arDivider = math.ceil(len(inputList)/maxArgs)
  npInputList = np.array(inputList)
  inputListPart = np.array_split(npInputList, arDivider)
  i=0;
  for part in inputListPart:
    filelist = "-v "+str(vol)+" "+strList(part.tolist())
    partOutFile = str(newDir)+"pt"+str(i)+".wav"
    !sox {sox_verbose} {filelist} -r 48000 -c 2 -b 24 {partOutFile}
    i+=1
  allParts = glob(newDir+"*.wav")
  input ="-v "+str(vol)+" "+strList(allParts)
  return input
  
drive_dir = stripPath(drive_dir, True)
output_dir = stripPath(output_dir, True)

if (output_dir is "" or output_dir is "/") and save_to_drive is True:
  if multitrack is True:
    xresults = glob(drive_dir+"/*/")
    for xresult in xresults:
      if not os.path.isdir(xresult+"/result"):
        !mkdir "{xresult}result"
  else:
    if not os.path.isdir(drive_dir+"/result"):
      !mkdir "{drive_dir}/result"

In [ ]:
#@title Initialize
hardReset = True
skipSliceLengths = False
skipConversion = False
skipChopping = False

### Butchery

In [ ]:
#@title Butchery

if hardReset is True:
  veryHardReset()

if smooth_wave is True:
  skipSliceLengths = True
  sliceLengths = [slices_per_beat]

###

track_inputs = glob(dir_tracks+"*/")
track_inputs.sort()
ti = 0
for track in track_inputs:
  trackIter = ti+1

  # General
  input_dir = track
  dir_conv = str(track)+"conv/"
  dir_conv_tmp = str(track)+"conv_tmp/"
  dir_chops = str(track)+"chop/"
  dir_parts = str(track)+"parts/"
  dir_result = str(track)+"result/"
  if not os.path.isdir(dir_chops):
    !mkdir {dir_chops}
  if not os.path.isdir(dir_parts):
    !mkdir {dir_parts}

  # Basics
  if smooth_wave is True:
    # Smooth Wave
    if crossfade is True:
      title = "Constant fading pattern"
    else:
      title = "Constant pattern"
    if preset is not "None":
      title = preset
    bpm = global_bpm
    totalSlices = math.ceil((bpm/min)*slices_per_beat*approx_duration)
    trackDuration = approx_duration
    pretty_trackDuration = str(datetime.timedelta(seconds=trackDuration))
    slice = min/bpm/slices_per_beat
  else:
    # Use relevant trackData
    title = trackData[ti][0]
    if trackData[ti][1] > 0:
      bpm = trackData[ti][1]
    else:
      bpm = global_bpm
    slice = min/bpm
    sectionRepeat = trackData[ti][2]
    sectionContent = trackData[ti][3]
    trackDuration = sum(sectionRepeat)*4*slice
    pretty_trackDuration = str(datetime.timedelta(seconds=trackDuration))
  beat = min/bpm
  materialDuration = getDuration(track, oneshots, slice)
  pretty_materialDuration = str(datetime.timedelta(seconds=materialDuration))
  print('\n>>> GENERATE TRACK', trackIter, ':', title, '\n')
  print('BPM:', bpm)
  if smooth_wave is True:
    print('Track target duration:', pretty_trackDuration)
  else:
    print('Track duration:', pretty_trackDuration)
  print('Source material duration:', pretty_materialDuration)
  print('Pulse duration:', beat)

  conflictWarn()

  if crossfade is True and use_patterns is True:
    print('\nWARN!\nPatterns (trackData[]) will be ignored because crossfade was checked. Uncheck crossfade to use patterns.')
    
  # Replicate source material if track is longer than source material combined
  sourceMultiplier = math.ceil(trackDuration/materialDuration);
  multiplyMaterial = False
  mmd = False
  if trackDuration > materialDuration and oneshots is False:
    #exit('FATAL: Track duration is greater than the duration of all input material combined. Unable to proceed.')
    mmd = True
    print('\nWARN!\nTrack duration is greater than the duration of source material.\nSouce material will be repeated '+str(sourceMultiplier)+' times.')
  if mmd is True or oneshots is True:
    multiplyMaterial = True

  # Define chops used in currently processed track
  if skipSliceLengths is False:
    sliceLengths = []
    print('\n> Determine used prime cuts')
    for i in range(len(sectionRepeat)):
      for sliceLen in sectionContent[i][0]:
        #sliceCounts[sliceLen] += 1
        if sliceLen > 0 and sliceLen not in sliceLengths:
          if not os.path.isdir(dir_chops+str(sliceLen)):
            !mkdir {dir_chops}/{sliceLen}
          sliceLengths.append(sliceLen)
          #print('Slice to', sliceLengths)
    sliceLengths.sort()
    print('Chop to', sliceLengths)

  if smooth_wave is True:
    !mkdir {dir_chops}/{slices_per_beat}

  # Convert material
  if os.path.isdir(dir_conv+"1"):
    skipConversion = True
  if skipConversion is False:
    allfiles = []
    allfiles = listFiles(input_dir)
    print('\n> MARINATE')
    !mkdir {dir_conv}
    process(allfiles, dir_conv, "convert", oneshots, silence_handler)

  # Butcher & shuffle
  if skipChopping is False:
    # Make some silence
    print('Generate some silence...', end=' ')
    halfBeat = slice/2
    silencio = dir_chops+"silencio.wav"
    silencioh = dir_chops+"silencioh.wav"
    silencio3s = dir_chops+"silencio-3s.wav"
    silencio5s = dir_chops+"silencio-5s.wav"
    !sox {sox_verbose} -n -r 48000 -c 2 {silencio} trim 0.0 {beat}
    !sox {sox_verbose} -n -r 48000 -c 2 {silencioh} trim 0.0 {halfBeat}
    !sox {sox_verbose} -n -r 48000 -c 2 {silencio3s} trim 0.0 3
    !sox {sox_verbose} -n -r 48000 -c 2 {silencio5s} trim 0.0 5
    print('Done.\n')
    # Do the chop chop
    print('>> BUTCHER')
    chopLists = [None] * (max(sliceLengths) + 1)
    chopListCounts = [0] * (max(sliceLengths) + 1)    
    for sliceLength in sliceLengths:
      slice = beat/sliceLength
      dir_subChops = str(dir_chops)+str(sliceLength)+"/"
      if oneshots is True:
        mode = 'cut'
      elif smooth_wave is True and crossfade is True:
        mode = 'smooth'
      else:
        mode = 'slice'
      filelisttest = [f for f in listdir(dir_conv) if isfile(join(dir_conv, f))]
      chop(dir_conv, dir_subChops, [f for f in listdir(dir_conv) if isfile(join(dir_conv, f))], slice, fadeIn, fadeOut, mode, tremolo)
      if oneshots is False:
        removeResidue(dir_subChops)
      allChops = glob(dir_subChops+"*.wav")
      random.shuffle(allChops)
      chopLists[sliceLength] = allChops
      if multiplyMaterial is True:
        for _ in range(sourceMultiplier):
          random.shuffle(allChops)
          chopLists[sliceLength].extend(allChops)

  # Concatenate  
  inputList = []
  print('\n> DEFINE CONCATENATION')
  if smooth_wave is True:
    trackLength = int(totalSlices)
    if len(chopLists[slices_per_beat][:trackLength]) > maxArgs:
      print('Bash is unable to keep track of all the butcher\'s chops, stitching up some chunks...')
      input = stitchChunk(chopLists[slices_per_beat][:trackLength], dir_parts)
      print('Done.')
    else:
      input = "-v "+str(vol)+" "+strList(chopLists[slices_per_beat][:trackLength])
    if crossfade is True:
      pad = slice/2
      # Create 2 tracks for padded merging. Dirty but works.
      if len(chopLists[slices_per_beat][trackLength:trackLength*2]) > maxArgs:
        input2 = stitchChunk(chopLists[slices_per_beat][trackLength:trackLength*2], dir_parts)
      else:
        input2 = "-v "+str(vol)+" "+strList(chopLists[slices_per_beat][trackLength:trackLength*2])
      !sox {sox_verbose} {input} {silencio5s} -r 48000 -c 2 -b 24 /content/tmp/sw1.wav 
      !sox {sox_verbose} {input2} {silencio5s} -r 48000 -c 2 -b 24 /content/tmp/sw2.wav pad {pad} 0
      input = "/content/tmp/sw1.wav /content/tmp/sw2.wav"
  else:
    # List all required slices
    for i in range(len(sectionRepeat)):
      print("Baron", i, "prime cuts", sectionContent[i][0])
      for x in range(sectionRepeat[i]):
        for sliceLen in sectionContent[i][0]:
          sliceLen = int(sliceLen)
          if sliceLen > 0:
            for y in range(sliceLen):
              currentChopList = chopLists[sliceLen]
              fileNo = chopListCounts[sliceLen]
              if oneshots is True:
                inputList.append(random.choice(currentChopList))
              else:
                inputList.append(currentChopList[fileNo])
              chopListCounts[sliceLen] += 1
          else:
            inputList.append(silencio)
    if len(inputList) > maxArgs:
      print('Bash is unable to keep track of all the butcher\'s chops, stitching up some chunks...')
      input = stitchChunk(inputList, dir_parts)
      print('Done.')
    else:
      input = "-v "+str(vol)+" "+strList(inputList)
  print('Done.')

  # Render
  if material_from is "googleDrive":
    if save_to_drive is True:
      saveDir = output_dir
    else:
      saveDir = dir_result
      !mkdir {saveDir}
    if multitrack is True:
      saveDir = saveDir+str(trackIter)
      !mkdir "{saveDir}"
    #  output = saveDir+"/butcherymonster-"+str(trackIter)+"-"+str(bpm)+"bpm-"+rndStr(8)+".wav"
    #else:
    #  #output = drive_dir+"/result/butcherymonster-"+str(trackIter)+"-"+str(bpm)+"bpm-"+rndStr(8)+".wav"
    #  output = saveDir+"butcherymonster-"+str(trackIter)+"-"+str(bpm)+"bpm-"+rndStr(8)+".wav"
    output = saveDir+"/"+prefix+"-"+str(trackIter)+"-"+str(bpm)+"bpm-"+rndStr(8)+".wav"
  else:
    output = dir_tmp+str(trackIter)+".wav"
  display_output = output.replace(saveDir, '')
  print('\n> STITCH ALL THOSE PRIME CUTS TOGETHER LIKE MOTHERFUCKING FRANKENSTEIN')
  pitch_out = ""
  if smooth_wave is True and crossfade is True:
    merge = "-m"
  else:
    merge = ""
  print('Save file to', display_output, '...', end=' ')
  !sox {sox_verbose} {merge} {input} {silencio5s} -r 48000 -c 2 -b 24 "{output}" pitch {pitch} reverb {reverb_amount} {reverb_damping} 100 100 0 0
  print('Done.')
  print('Encode preview clips...', end=' ')
  preview_output = dir_preview+"preview-"+str(trackIter)+".mp3"
  !ffmpeg {ffmpeg_verbose} -y -i "{output}" -b:a 320k -ss 0 -t 120 {preview_output}
  print('Done.')
  print('\n>>> TRACK', trackIter, 'FINISHED\n\n')  
  
  ti+=1

# # Previews
# !ffmpeg -hide_banner -loglevel panic -y -i "{output}" -b:a 320k -ss 0 -t 120 {dir_preview}/preview.mp3
# !ffmpeg -hide_banner -loglevel panic -y -i "{output}" -ss 0 -t 120 {dir_preview}/preview.wav
# baselen = 60/bpm
# kicklen1 = baselen*0.75
# kicklen2 = baselen*1.25
# kick1 = "/content/tmp/slicer/kick-punchy.wav"
# !sox {kick1} {dir_preview}/trimmed1a-punchy.wav trim 0 {kicklen1}
# !sox {kick1} {dir_preview}/trimmed1b-punchy.wav trim 0 {kicklen2}
# !sox {kick1} {dir_preview}/trimmed2-punchy.wav trim 0 {baselen}
# !sox {dir_preview}/trimmed1a-punchy.wav {dir_preview}/trimmed1b-punchy.wav {dir_preview}/block.wav
# repeatCount = bpm*2
# !sox {dir_preview}/block.wav /{dir_preview}/loop1-punchy.wav repeat {bpm}
# !sox {dir_preview}/trimmed2-punchy.wav {dir_preview}/loop2-punchy.wav repeat {repeatCount}
# !sox -m -v 0.45 {dir_preview}/loop1-punchy.wav "{dir_preview}/preview.wav" {dir_preview}/out-punchy-mix1.wav
# !sox -m -v 0.45 {dir_preview}/loop2-punchy.wav "{dir_preview}/preview.wav" {dir_preview}/out-punchy-mix2.wav
# !ffmpeg -hide_banner -loglevel panic -y -i {dir_preview}/out-punchy-mix1.wav -b:a 320k -ss 0 -t 120 {dir_preview}/preview1-punchy.mp3
# !ffmpeg -hide_banner -loglevel panic -y -i {dir_preview}/out-punchy-mix2.wav -b:a 320k -ss 0 -t 120 {dir_preview}/preview2-punchy.mp3
# player(dir_preview+'/preview1-punchy.mp3')
# player(dir_preview+'/preview2-punchy.mp3')

In [ ]:
#@title Preview
wfAudio(dir_preview+"/preview-1.mp3", output)